D:\All Acads\3-2\Information Retrival\Assignment Files\Document_Corpse

In [49]:
import os
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.metrics.distance import edit_distance
from nltk.corpus import words
import time

In [50]:
def rotate(str, n):
    """
    Left rotates the given string by n times
    :param str: the string that needs to be rotated
    :param n: the number of times the string needs to be left rotated
    :returns: the rotated string
    """
    return str[n:] + str[:n]

In [51]:
def AND(list1, list2):
    """
    performs intersection of two given lists.
    
    :param list1: list of docIDs
    :param list2: list of docIDs
    :returns: intersection of the two given lists
    """
    final_list=[]
    for docID in list1:
        if docID in list2:
            final_list.append(docID)
    return final_list

In [52]:
def OR(list1, list2):
    """
    performs union of two given lists.
    
    :param list1: list of docIDs
    :param list2: list of docIDs
    :returns: union of the two given lists
    """
    final_list = list(set(list1) | set(list2))
    return final_list

In [53]:
def NOT(list1):
    """
    returns the items that are not there in this list
    
    :param list1: list of docIDs
    :returns: list of items that are not present in this list
    """
    total_list = list(range(42))
    return [i for i in total_list if i not in list1]

In [54]:
def removeStopWords(listOfWords):
    """
    Removes the stop words that have been scrapped from NLTK library.
    
    :param listOfWords: The total list of words present in the document
    :returns: The list in which the stopwords have been removed
    """
    stop_words = set(stopwords.words('english'))
    return [word.lower() for word in listOfWords if not word.lower() in stop_words]

In [55]:
def stemming(listOfWords):
    """
    Performs stemming on each of the given list of words
    
    :param listOfWords: a list of words on which the stemming needs to be performed
    :returns: a list of stemmed words
    """
    ps = PorterStemmer()
    return [ps.stem(w) for w in listOfWords]

In [56]:
def correction(word, all_words):
    """
    Corrects the given word if it has a spelling mistake. The list of correct words is  scrapped from NLTK library
    
    :param word: the word that might need correction
    :returns: corrected word if the given word is spelled wrong
    """
    correct_words = all_words

    all_edit_distances = [(edit_distance(word, w),w) for w in correct_words if w[0]==word[0]]
    return sorted(all_edit_distances, key = lambda val:val[0])[0][1]


In [57]:
def wildCardSearch(word, permutermIndex, invertedIndex):
    """
    wildcard query terms can have many possibilites based on the document corpse.
    This function takes the wildcard terms and searches for all the possiblites of the word and returns a combined list of 
    documents that the possible words are present
    
    :param word: the wildcard query term
    :param permutermIndex: the permuterm index which has all the words with their possible left rotations
    :param invertedIndex: the inverted index for searching the possible word postings
    :returns: the postings of union of all possible words
    """
    possibilties = []
    dword = word + '$'
    while dword[-1] != '*':
        dword = rotate(dword, -1)
    dword = dword[:-1]
    for word, permuterms in permutermIndex.items():
        for permutern in permuterms:
            if permutern.find(dword) == 0:
                possibilties.append(word)
                break
    finalList = []
    for word in possibilties:
        finalList = OR(finalList, invertedIndex[word])
    return finalList

In [58]:
def replaceByLists(listOfTokens, invertedIndex, permutermIndex, all_words):
    """
    On tokenizing the query, we will have the query terms present in a list of tokens. These can be replaced by their postings
    and this function performs that
    
    :params listOfTokens: the list of tokens in the query
    :params invertedIndex: the inverted index data structure
    :params permutermIndex: the permuterm index data structure
    """
    for index, token in enumerate(listOfTokens):
        if token not in ['&', '~', '|', "(", ")"]:
            if '*' in token:
                listOfTokens[index] = wildCardSearch(token, permutermIndex, invertedIndex)
            else:
                token = token.lower()
                token = correction(token, all_words)
                token = stemming([token])[0]
                listOfTokens[index] = invertedIndex[token]
    return listOfTokens

In [59]:
def resolveSubProblem(listOfTokens):
    """
    resloves tokens of the form list1 operator list2 where each list1  and list2 are postings while operator can be AND or OR
    
    :params listOfTokens: the tokens of the subproblem considered
    """
    resolved = []
    while '~' in listOfTokens:
        indexNot = listOfTokens.index('~')
        negatedList = NOT(listOfTokens[indexNot+1])
        listOfTokens = listOfTokens[:indexNot] + listOfTokens[indexNot+2:]
        listOfTokens.insert(indexNot, negatedList)
    if listOfTokens[1] == "&":
        resolved = AND(listOfTokens[0], listOfTokens[2])
    else:
        resolved = OR(listOfTokens[0], listOfTokens[2])
    return resolved

In [60]:
def updateInvertedIndex(invertedIndex, wordsList, docID):
    """
    Updates the inverted Index with the given new list of words
    
    :param permutermIndex: the old inverted index
    :param wordsList: list of new words to be indexed
    :param docID: ID of the document from which these words have been taken
    :returns: the updated inverted index
    """
    for word in wordsList:
        if word not in invertedIndex:
            invertedIndex[word] = [docID]
        else:
            if invertedIndex[word][-1] != docID:
                invertedIndex[word].append(docID)
    return invertedIndex

In [61]:
def updatePermutermIndex(permutermIndex, wordsList):
    """
    Updates the permuterm Index with the given new list of words
    
    :param permutermIndex: the old permuterm index
    :param wordsList: list of new words to be indexed
    :returns: the updated permuterm list
    """
    for word in wordsList:
        if word not in permutermIndex:
            dkey = word + "$"
            permutermIndex[word] = []
            for i in range(len(dkey),0,-1):
                permutermIndex[word].append(rotate(dkey,i))
    return permutermIndex

In [62]:
def preProcessAndBuild(documentNames, corpseLocation):
    """
    Performs pre procressing techniques on the given dataset/document corpse and builds inverted index and permuterm index
    Pre-processing techniques performed:
        1. Stopword removal
        2. Stemming
        3. Miscellaneous. Includes removing of non significant symbols like =, /, and .
    
    :param documentNames: The list of document names that this system will interact with
    :param corpseLocation: The absolute location of the folder in which all the documents are present
    :returns: Two data structures. Inverted index and Permuterm index
    """
    
    invertedIndex = {}
    permutermIndex = dict()
    all_words = []
    for docID, doc in enumerate(documentNames):
        # open the document and read its contents
        document = open(corpseLocation + "\\" + doc)
        doc_data = document.read()
        document.close()
        
        # Remove miscellaneous symbols
        doc_data = doc_data.replace("=", "")
        doc_data = doc_data.replace(".", " ")
        doc_data = doc_data.replace("/", " ")
        
        # Tokenize the document contents for preprocessing
        words = word_tokenize(doc_data)
        
        # Remove stop words
        filtered_words = removeStopWords(words)
        
        all_words += filtered_words
        
        # Perform stemming on each word
        stemmed_words =  stemming(filtered_words)
        
        # Update the indexes with the new set of words
        invertedIndex = updateInvertedIndex(invertedIndex, stemmed_words, docID)
        permutermIndex = updatePermutermIndex(permutermIndex, stemmed_words)
        
    return (invertedIndex, permutermIndex, all_words)
    

Driver Code

In [63]:
# Preprocessing
path_to_doc_corpse = input("Enter the absolute path to the document corpse")
os.chdir(path_to_doc_corpse)
documentNames = [doc for doc in os.listdir() if doc.endswith(".txt")]

print("Pre-processing and building index please wait...")
start_time = time.time()
invertedIndex, permutermIndex, all_words = preProcessAndBuild(documentNames, path_to_doc_corpse)
end_time = time.time()
print("Pre-processing done!")
sec = end_time-start_time
mins = sec // 60
sec = sec % 60
hours = mins // 60
mins = mins % 60
print("Time Lapsed = {0}:{1}:{2}".format(int(hours),int(mins),sec))

Enter the absolute path to the document corpseD:\All Acads\3-2\Information Retrival\Assignment Files\Document_Corpse
Pre-processing and building index please wait...
Pre-processing done!
Time Lapsed = 0:0:17.011948823928833


In [64]:
# Query cell
query = input("Enter your query")
queryTokens = query.split()
# print(queryTokens)
queryTokens = replaceByLists(queryTokens, invertedIndex, permutermIndex, all_words)
while len(queryTokens) != 1:
    i = 0
    j = 0
    for j in range(len(queryTokens)):
        if queryTokens[j] == ')':
            i = j-1
            while queryTokens[i] != '(':
                i -= 1
            resolved = resolveSubProblem(queryTokens[i+1:j])
            queryTokens = queryTokens[:i] + queryTokens[j+1:]
            queryTokens.insert(i, resolved)
            break
queryTokens[0]

Enter your query( paul & dream )


[0,
 1,
 2,
 3,
 5,
 6,
 8,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40]